In [1]:
import pandas as pd
import numpy as np
import tsfresh as ts
from dateutil.relativedelta import relativedelta
import seaborn as sns
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh import extract_features, extract_relevant_features, select_features
import pandas as pd
import numpy as np

In [2]:
meteo = pd.read_csv('meteo.csv')

In [3]:
meteo['d'] = pd.to_datetime(meteo['d'])

In [4]:
meteo.index = meteo['d']

In [5]:
meteo.columns = ['mintemp', 'meantemp', 'maxtemp', 'prec', 'd']

In [6]:
def neg_sum(array):
    return sum([x for x in array if x < 0])

def plus_sum(array):
    return sum([x for x in array if x > 0])

def neg_sum_squared(array):
    return sum([x**2 for x in array if x < 0])

def plus_sum_squared(array):
    return sum([x**2 for x in array if x > 0])

# Функция для вычисления количества переходов через ноль
def calculate_number_of_zeros(series):
    return ((series[:-1] * series[1:]) < 0).sum()

# Функция для вычисления среднего изменения
def calculate_mean_change(series):
    return np.mean(np.diff(series))

In [7]:
def extract_monthly_features(df):
    monthly_features = df.resample('M').apply({
        'meantemp': ['mean'],
        'mintemp': ['mean', 'sum'],
        'maxtemp': ['mean', 'sum'],
        'prec': ['sum', 'max', lambda x: (x > 1.0).sum(), lambda x: (x > 10).sum()],
    })
    
    monthly_features.columns = [
        'Средняя температура', 'Минимальная температура', 'Сумма минимальных температур',
        'Максимальная температура', 'Сумма максимальных температур', 'Сумма осадков',
        'Максимальное количество осадков', 'Количество дней с суточными осадками более 1 мм', 'Количество дней с суточными осадками более 10 мм'
    ]
    
    monthly_features['Разница между максимальной и минимальной температурой '] = df['maxtemp'].resample('M').max() - df['mintemp'].resample('M').min()
    monthly_features['mean_temp_amplitude'] = (df['maxtemp'] - df['mintemp']).resample('M').mean()
    monthly_features['Количество дней со среднесуточной температурой ниже 0°С'] = df['mintemp'].resample('M').apply(lambda x: (x < 0).sum())
    monthly_features['Количество дней со среднесуточной температурой выше 0°С'] = df['maxtemp'].resample('M').apply(lambda x: (x > 0).sum())
    monthly_features['Количество дней с среднесуточной отрицательной температурой и наличием осадков'] = df.resample('M').apply(lambda x: ((x['meantemp'] < 0) & (x['prec'] > 0)).sum())
    monthly_features['Сумма отрицательных температур'] = df['meantemp'].resample('M').apply(neg_sum)
    monthly_features['Сумма положительных температур'] = df['meantemp'].resample('M').apply(plus_sum)
    monthly_features['Сумма квадратов отрицательных температур'] = df['meantemp'].resample('M').apply(neg_sum_squared)
    monthly_features['Сумма квадратов положительных температур'] = df['meantemp'].resample('M').apply(plus_sum_squared)
    monthly_features['Сумма осадков в дни с отрицательными температурами'] = df.resample('M').apply(lambda x: x[(x['meantemp'] < 0)]['prec'].sum())
    monthly_features['Сумма квадратов среднесуточных температур'] = df['meantemp'].resample('M').apply(lambda x: np.sum(x**2))
    monthly_features['Сумма квадратов суточного количества осадков'] = df['prec'].resample('M').apply(lambda x: np.sum(x**2))
    monthly_features['Количество переходов через 0°С'] = df['meantemp'].resample('M').apply(calculate_number_of_zeros)
    monthly_features['Среднее изменение среднесуточных температур'] = df['meantemp'].resample('M').apply(calculate_mean_change)
    monthly_features['Среднее изменение суточного количества осадков'] = df['prec'].resample('M').apply(calculate_mean_change)
    
    
    return monthly_features

In [8]:
dataset = extract_monthly_features(meteo)

In [9]:
dataset_month_1 = dataset.shift(1).add_suffix('_1 месяц назад')
dataset_month_2 = dataset.shift(2).add_suffix('_2 месяца назад')
dataset_month_3 = dataset.shift(3).add_suffix('_3 месяца назад')
dataset_month_4 = dataset.shift(4).add_suffix('_4 месяца назад')
dataset_month_5 = dataset.shift(5).add_suffix('_5 месяца назад')
dataset_month_6 = dataset.shift(6).add_suffix('_6 месяцев назад')
dataset_month_7 = dataset.shift(7).add_suffix('_7 месяцев назад')
dataset_month_fut_1 = dataset.shift(-1).add_suffix('_в следующем месяце')
dataset_month_fut_2 = dataset.shift(-2).add_suffix('_через 2 месяца')
dataset_month_fut_3 = dataset.shift(-3).add_suffix('_через 3 месяца')

In [10]:
dataset = pd.concat([dataset, dataset_month_1, dataset_month_2, dataset_month_3, dataset_month_4, dataset_month_5, dataset_month_6, dataset_month_7, dataset_month_fut_1, 
                     #dataset_month_fut_2, dataset_month_fut_3
                    ], axis=1)

In [11]:
d = []
for i in range(1926, 2023, 1):
    for j in range(1, 13, 1):
        c = str(i)+'-'+str(j)
        d.append(c)
data = {'date': d}
df = pd.DataFrame(data)
ij = pd.read_csv("ij.csv")
df = df.merge(ij, how='left', on='date').fillna(0)
df['Datetime'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m')

In [12]:
dataset['Datetime'] = pd.to_datetime(dataset.index)
dataset['Datetime'] = dataset['Datetime'].dt.strftime('%Y-%m')

In [13]:
dataset = df.merge(dataset, on='Datetime', how='left').drop(['date'], axis=1)

In [14]:
dataset['Datetime'] = pd.to_datetime(dataset['Datetime'])

In [15]:
dataset['month'] = dataset['Datetime'].dt.month

In [16]:
for i in dataset.columns[2:]:
    dataset[i] = dataset.groupby(['month', 'ij'])[i].transform(lambda x: x.fillna(x.median()))

In [17]:
dataset['ij_1'] = dataset['ij'].shift(-1)
dataset['ij_2'] = dataset['ij'].shift(-2)
dataset['ij_3'] = dataset['ij'].shift(-3)
dataset.fillna(0, inplace=True)

C:\Users\s.iglin\AppData\Local\Temp\ipykernel_22788\3302444362.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['ij_1'] = dataset['ij'].shift(-1)
C:\Users\s.iglin\AppData\Local\Temp\ipykernel_22788\3302444362.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['ij_2'] = dataset['ij'].shift(-2)
C:\Users\s.iglin\AppData\Local\Temp\ipykernel_22788\3302444362.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

In [18]:
dataset = dataset.drop(['ij'], axis=1)

In [19]:
dataset

,Datetime,Средняя температура,Минимальная температура,Сумма минимальных температур,Максимальная температура,Сумма максимальных температур,Сумма осадков,Максимальное количество осадков,Количество дней с суточными осадками более 1 мм,Количество дней с суточными осадками более 10 мм,...,Сумма осадков в дни с отрицательными температурами_в следующем месяце,Сумма квадратов среднесуточных температур_в следующем месяце,Сумма квадратов суточного количества осадков_в следующем месяце,Количество переходов через 0°С_в следующем месяце,Среднее изменение среднесуточных температур_в следующем месяце,Среднее изменение суточного количества осадков_в следующем месяце,month,ij_1,ij_2,ij_3
0,1926-01-01,-16.500000,-21.400000,-663.4,-11.800000,-365.8,9.3,0.3,0,0,...,5.6,7078.680000,1.12,0.0,0.000000,0.000000e+00,1,0.0,0.0,0.0
1,1926-02-01,-15.900000,-20.800000,-582.4,-11.600000,-324.8,5.6,0.2,0,0,...,6.2,4614.040000,1.24,0.0,0.000000,0.000000e+00,2,0.0,0.0,0.0
2,1926-03-01,-12.200000,-17.700000,-548.7,-7.300000,-226.3,6.2,0.2,0,0,...,3.0,874.800000,0.30,0.0,0.000000,0.000000e+00,3,0.0,0.0,0.0
3,1926-04-01,-5.400000,-9.800000,-294.0,-1.000000,-30.0,3.0,0.1,0,0,...,0.0,0.000000,0.31,0.0,0.000000,0.000000e+00,4,0.0,0.0,0.0
4,1926-05-01,0.000000,-2.400000,-74.4,3.000000,93.0,3.1,0.1,0,0,...,0.0,1598.700000,0.00,0.0,0.000000,0.000000e+00,5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,2022-08-01,15.367339,10.667742,330.7,19.519355,605.1,39.6,16.0,5,1,...,0.0,1624.558906,401.02,0.0,-0.256897,0.000000e+00,8,0.0,0.0,0.0
1160,2022-09-01,6.211250,3.646667,109.4,8.963333,268.9,62.0,10.5,11,1,...,11.5,676.058906,239.69,0.0,-0.061667,-6.666667e-02,9,0.0,0.0,0.0
1161,2022-10-01,3.021371,1.116129,34.6,4.596774,142.5,53.7,8.0,14,0,...,30.2,1648.339219,85.29,0.0,-0.309483,-7.241379e-02,10,0.0,0.0,0.0
1162,2022-11-01,-6.265417,-9.200000,-276.0,-3.726667,-111.8,30.9,5.0,9,0,...,10.7,6903.979066,9.39,0.0,-0.329167,-3.700743e-18,11,0.0,0.0,0.0


In [20]:
dataset.to_csv('dataframe.csv', index=False)